In [2]:
!nvidia-smi

Tue Oct  7 17:47:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        On  |   00000000:81:00.0 Off |                  N/A |
|  0%   40C    P8             31W /  370W |       2MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!kill 584175

In [2]:
# Block 1️⃣ – Model Setup

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

def setup_model(model_id="mistralai/Mistral-7B-Instruct-v0.3"):
    """
    Load Mistral-7B-Instruct and prepare a text-generation pipeline.
    """
    torch.backends.cudnn.benchmark = True  # GPU speed-up
    
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.float16,
    )
    model.config.use_cache = False  # disable cache for long prompts

    generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device_map="auto"
    )
    return generator, tokenizer

# Initialize model & tokenizer
text_pipe, tokenizer = setup_model("mistralai/Mistral-7B-Instruct-v0.3")

# quick sanity check
print(text_pipe)


/upb/users/b/balram/profiles/unix/cs/.conda/envs/gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.09it/s]
Device set to use cuda:0


In [9]:
# Block 2️⃣ – Prompt Creation (Reason → Verify → Final Triples; single block incl. ontology + extra)

from textwrap import dedent

def _concept_label(ontology_json, qid):
    return next((c["label"] for c in ontology_json["concepts"] if c["qid"] == qid), "")

def format_ontology_concepts(ontology_json):
    return ", ".join(c["label"] for c in ontology_json["concepts"])

def format_ontology_relations(ontology_json):
    lines = []
    for r in ontology_json["relations"]:
        dom = _concept_label(ontology_json, r["domain"])
        rng = _concept_label(ontology_json, r["range"])
        lines.append(f'- {r["label"]}({dom},{rng})')
    return "\n".join(lines)

def build_reason_then_extract_prompt(
    ontology_json,
    test_sentence,
    worked_example=None,
    allow_light_norm=True,
    relation_cues_text=None  # optional Step 0 paraphrase cues (can be left None)
):
    concepts_line  = format_ontology_concepts(ontology_json)
    relations_block = format_ontology_relations(ontology_json)
    norm_note = "normalize only trivial cases (e.g., Japanese→Japan)" if allow_light_norm else "copy spans verbatim; no normalization"

    step0_block = ""
    if relation_cues_text:
        step0_block = dedent(f"""
        Step 0 (Paraphrase expansion for this sentence):
        {relation_cues_text.strip()}
        """)

    example_block = ""
    if worked_example:
        example_block = dedent(f"""
        ### WORKED EXAMPLE
        Sentence:
        {worked_example["sentence"]}

        Step 1 (Entities & types):
        {worked_example["step1"]}

        Step 2 (Verify relations):
        {worked_example["step2"]}

        ### FINAL TRIPLES
        {worked_example["final_triples"].strip()}
        """).strip()

    rules = dedent("""
    RULES:
    - Prefer relation labels from ONTOLOGY RELATIONS when they match the evidence.
    - If the sentence clearly states another SPO relation that is NOT in the ontology, still include it using a concise predicate phrase derived from the trigger words in the sentence (do NOT invent facts).
    - Resolve simple coreference (e.g., "the film", "this movie", "it" → the film title in this sentence).
    - Quote/mention evidence in Step 2 for each emitted triple.
    - Output one triple per line, format: predicate("Subject","Object").
    - Base decisions ONLY on the TEST SENTENCE.
    - Avoid duplicates; apply light normalization only (e.g., Japanese→Japan).
    """)

    prompt = dedent(f"""
    TASK: Extract all SPO triples that the TEST SENTENCE clearly supports.
    - Use the ontology to guide relation labeling when possible.
    - {norm_note}.
    - If a valid SPO fact has no matching ontology label, still output it (single FINAL TRIPLES list).

    ONTOLOGY CONCEPTS:
    {concepts_line}

    ONTOLOGY RELATIONS (argument types):
    {relations_block}

    {rules}

    {step0_block}

    {example_block if example_block else ""}

    ### TEST SENTENCE
    "{test_sentence}"

    Step 1 (Entities & types):
    Step 2 (Verify relations): For each candidate triple, quote or mention the trigger phrase and spans.

    ### FINAL TRIPLES
    # one triple per line; include ontology-mapped AND extra sentence-backed relations
    """).strip()

    return prompt


# --- Example ontology + text ---------------------------------------

input_text = """The Life on the Earth also known as Tracing the Gray Summer, is a 2001 Japanese anime drama film by Satoshi Dezaki, a joint production between the Magic Bus and the GoGo Visual Planning which recounts the true story of the Seveso disaster, a chemical incident occurred over the Italian town of Seveso in 1976, which is still considered one of the worst ecological disasters in history."""

ontology = {
    "title": "Movie Ontology",
    "id": "ont_1_movie",
    "concepts": [
        {"qid": "Q5", "label": "human"},
        {"qid": "Q515", "label": "city"},
        {"qid": "Q6256", "label": "country"},
        {"qid": "Q11424", "label": "film"},
        {"qid": "Q201658", "label": "film genre"},
        {"qid": "Q483394", "label": "genre"},
        {"qid": "Q1762059", "label": "film production company"},
        {"qid": "Q4220917", "label": "film award"},
        {"qid": "Q618779", "label": "award"},
        {"qid": "Q47461344", "label": "written work"},
        {"qid": "Q15773347", "label": "film character"},
        {"qid": "Q104649845", "label": "film organization"}
    ],
    "relations": [
        {"pid": "P57", "label": "director", "domain": "Q11424", "range": "Q5"},
        {"pid": "P58", "label": "screenwriter", "domain": "Q11424", "range": "Q5"},
        {"pid": "P136", "label": "genre", "domain": "Q11424", "range": "Q483394"},
        {"pid": "P144", "label": "based on", "domain": "Q11424", "range": "Q47461344"},
        {"pid": "P161", "label": "cast member", "domain": "Q11424", "range": "Q5"},
        {"pid": "P166", "label": "award received", "domain": "Q11424", "range": "Q618779"},
        {"pid": "P272", "label": "production company", "domain": "Q11424", "range": "Q1762059"},
        {"pid": "P495", "label": "country of origin", "domain": "Q11424", "range": "Q6256"},
        {"pid": "P577", "label": "publication date", "domain": "Q11424", "range": ""},
        {"pid": "P674", "label": "characters", "domain": "Q11424", "range": "Q15773347"},
        {"pid": "P840", "label": "narrative location", "domain": "Q11424", "range": "Q515"},
        {"pid": "P915", "label": "filming location", "domain": "Q11424", "range": "Q515"},
        {"pid": "P921", "label": "main subject", "domain": "Q11424", "range": ""},
        {"pid": "P1411", "label": "nominated for", "domain": "Q11424", "range": "Q618779"},
        {"pid": "P2130", "label": "cost", "domain": "Q11424", "range": ""}
    ]
}

worked_example = {
    "sentence": '"Resident Evil: Damnation is a 2012 Japanese animated film directed by Makoto Kamiya."',
    "step1": "- Resident Evil: Damnation → film\n- Makoto Kamiya → human\n- 2012 → year\n- Japanese → country → Japan\n- animated → genre",
    "step2": "- director(film,human): 'directed by Makoto Kamiya' ✓\n- country of origin(film,country): 'Japanese' → Japan ✓\n- publication date(film,year): '2012' ✓",
    "final_triples": 'director("Resident Evil: Damnation","Makoto Kamiya")\ncountry of origin("Resident Evil: Damnation","Japan")\npublication date("Resident Evil: Damnation","2012")'
}

prompt_text = build_reason_then_extract_prompt(
    ontology, input_text, worked_example=worked_example, allow_light_norm=True, relation_cues_text=None
)

print(prompt_text[:1200])


TASK: Extract all SPO triples that the TEST SENTENCE clearly supports.
    - Use the ontology to guide relation labeling when possible.
    - normalize only trivial cases (e.g., Japanese→Japan).
    - If a valid SPO fact has no matching ontology label, still output it (single FINAL TRIPLES list).

    ONTOLOGY CONCEPTS:
    human, city, country, film, film genre, genre, film production company, film award, award, written work, film character, film organization

    ONTOLOGY RELATIONS (argument types):
    - director(film,human)
- screenwriter(film,human)
- genre(film,genre)
- based on(film,written work)
- cast member(film,human)
- award received(film,award)
- production company(film,film production company)
- country of origin(film,country)
- publication date(film,)
- characters(film,film character)
- narrative location(film,city)
- filming location(film,city)
- main subject(film,)
- nominated for(film,award)
- cost(film,)


RULES:
- Prefer relation labels from ONTOLOGY RELATIONS when 

In [10]:
# Block 3️⃣ – Run Single Inference (chat template; continuation only)

def run_single_inference_chat(generator, tokenizer, base_prompt, max_new_tokens=1024):
    chat = [
        {"role": "system", "content": "You are a precise information-extraction model. Follow instructions carefully."},
        {"role": "user", "content": base_prompt}
    ]
    formatted = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

    print("🔹 Sending chat-formatted prompt to model ...")
    out = generator(
        formatted,
        max_new_tokens=max_new_tokens,
        temperature=0.3,
        top_p=0.9,
        do_sample=True,
        return_full_text=False,
        truncation=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )
    text = out[0]["generated_text"] if isinstance(out[0], dict) else out[0]
    print("\n✅ Model generation completed.\n")
    return text

# Run once
model_output = run_single_inference_chat(text_pipe, tokenizer, prompt_text, max_new_tokens=1024)

print("----- OUTPUT HEAD -----\n", model_output[:800])
print("\n----- OUTPUT TAIL -----\n", model_output[-800:])


🔹 Sending chat-formatted prompt to model ...

✅ Model generation completed.

----- OUTPUT HEAD -----
  Step 1 (Entities & types):
- The Life on the Earth (or Tracing the Gray Summer) → film
- Satoshi Dezaki → human
- 2001 → year
- Japanese → country → Japan
- anime → genre
- Magic Bus → film production company
- GoGo Visual Planning → film production company
- Seveso disaster → event
- Italian town of Seveso → city
- 1976 → year

Step 2 (Verify relations):
- director(film,human): 'by Satoshi Dezaki' ✓
- production company(film,film production company): 'a joint production between the Magic Bus and the GoGo Visual Planning' ✓
- based on(film,written work): 'which recounts the true story of the Seveso disaster' (implied, no direct mention)
- narrative location(film,city): 'Italian town of Seveso' ✓
- event occurred in(event,city): 'occurred over the Italian town of Seveso' ✓
- year of event(

----- OUTPUT TAIL -----
  Seveso' ✓
- event occurred in(event,city): 'occurred over the Italian 

In [3]:
# # =========================
# # Triple Extraction Pipeline (Steps 1–4)
# # - Single-call per input text
# # - Ontology-guided + open-set (include sentence-backed triples even if not in ontology)
# # - Processes first N inputs for fast debugging
# # =========================

# import os, json, re, time
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# # -------- Block 1: Model Setup --------
# def setup_model(model_id="mistralai/Mistral-7B-Instruct-v0.3"):
#     torch.backends.cudnn.benchmark = True
#     tokenizer = AutoTokenizer.from_pretrained(model_id)
#     model = AutoModelForCausalLM.from_pretrained(
#         model_id,
#         device_map="auto",
#         torch_dtype=torch.float16,
#     )
#     model.config.use_cache = False
#     generator = pipeline(
#         "text-generation",
#         model=model,
#         tokenizer=tokenizer,
#         device_map="auto"
#     )
#     return generator, tokenizer


# # -------- Block 2: Prompt Creation (Reason → Verify → Final Triples) --------
# from textwrap import dedent

# def _concept_label(ontology_json, qid):
#     return next((c["label"] for c in ontology_json.get("concepts", []) if c.get("qid") == qid), "")

# def format_ontology_concepts(ontology_json):
#     return ", ".join(c["label"] for c in ontology_json.get("concepts", []))

# def format_ontology_relations(ontology_json):
#     lines = []
#     for r in ontology_json.get("relations", []):
#         dom = _concept_label(ontology_json, r.get("domain"))
#         rng = _concept_label(ontology_json, r.get("range"))
#         lines.append(f'- {r["label"]}({dom},{rng})')
#     return "\n".join(lines)

# def build_reason_then_extract_prompt(
#     ontology_json,
#     test_sentence,
#     worked_example=None,
#     allow_light_norm=True,
#     relation_cues_text=None  # optional Step 0 paraphrase cues (keep None for now)
# ):
#     concepts_line   = format_ontology_concepts(ontology_json)
#     relations_block = format_ontology_relations(ontology_json)
#     norm_note = "normalize only trivial cases (e.g., Japanese→Japan)" if allow_light_norm else "copy spans verbatim; no normalization"

#     step0_block = ""
#     if relation_cues_text:
#         step0_block = dedent(f"""
#         Step 0 (Paraphrase expansion for this sentence):
#         {relation_cues_text.strip()}
#         """)

#     example_block = ""
#     if worked_example:
#         example_block = dedent(f"""
#         ### WORKED EXAMPLE
#         Sentence:
#         {worked_example["sentence"]}

#         Step 1 (Entities & types):
#         {worked_example["step1"]}

#         Step 2 (Verify relations):
#         {worked_example["step2"]}

#         ### FINAL TRIPLES
#         {worked_example["final_triples"].strip()}
#         """).strip()

#     rules = dedent("""
#     RULES:
#     - Prefer relation labels from ONTOLOGY RELATIONS when they match the evidence.
#     - If the sentence clearly states another SPO relation that is NOT in the ontology, still include it using a concise predicate phrase derived from the trigger words in the sentence (do NOT invent facts).
#     - Resolve simple coreference (e.g., "the university", "it", "this college" → the named university in this sentence).
#     - Quote/mention evidence in Step 2 for each emitted triple.
#     - Output one triple per line, format: predicate("Subject","Object").
#     - Base decisions ONLY on the TEST SENTENCE.
#     - Avoid duplicates; apply light normalization only where unambiguous.
#     """)

#     prompt = dedent(f"""
#     TASK: Extract all SPO triples that the TEST SENTENCE clearly supports.
#     - Use the ontology to guide relation labeling when possible.
#     - {norm_note}.
#     - If a valid SPO fact has no matching ontology label, still output it (single FINAL TRIPLES list).

#     ONTOLOGY CONCEPTS:
#     {concepts_line}

#     ONTOLOGY RELATIONS (argument types):
#     {relations_block}

#     {rules}

#     {step0_block}

#     {example_block if example_block else ""}

#     ### TEST SENTENCE
#     "{test_sentence}"

#     Step 1 (Entities & types):
#     Step 2 (Verify relations): For each candidate triple, quote or mention the trigger phrase and spans.

#     ### FINAL TRIPLES
#     # one triple per line; include ontology-mapped AND extra sentence-backed relations
#     """).strip()

#     return prompt


# # -------- Block 3: Single Inference (chat template; continuation-only) --------
# def run_single_inference_chat(generator, tokenizer, base_prompt, max_new_tokens=768, temperature=0.3):
#     chat = [
#         {"role": "system", "content": "You are a precise information-extraction model. Follow instructions carefully."},
#         {"role": "user", "content": base_prompt}
#     ]
#     formatted = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

#     out = generator(
#         formatted,
#         max_new_tokens=max_new_tokens,
#         temperature=temperature,
#         top_p=0.9,
#         do_sample=True,
#         return_full_text=False,
#         truncation=False,
#         eos_token_id=tokenizer.eos_token_id,
#         pad_token_id=tokenizer.eos_token_id,
#     )
#     return out[0]["generated_text"] if isinstance(out[0], dict) else out[0]


# # -------- Block 4: Extract & Parse FINAL TRIPLES --------
# FINAL_HEADER_REGEX = r'^\s*#{2,}\s*FINAL\s+TRIPLES\s*$'  # matches "## FINAL TRIPLES" or "### FINAL TRIPLES"

# def extract_final_triples_block(text: str) -> str:
#     # 1) primary: markdown header style
#     header = re.search(FINAL_HEADER_REGEX, text, re.IGNORECASE | re.MULTILINE)
#     if not header:
#         # 2) relaxed fallback: line containing "FINAL TRIPLES" at end
#         header = re.search(r'FINAL\s+TRIPLES\s*[:\-]*\s*$', text, re.IGNORECASE | re.MULTILINE)
#         if not header:
#             return ""
#     start = header.end()
#     tail = text[start:].strip()
#     # stop at next markdown header or end
#     nxt = re.search(r'^\s*#{2,}\s+[A-Z].*$', tail, re.MULTILINE)
#     if nxt:
#         tail = tail[:nxt.start()].strip()
#     return tail

# def parse_triples_block(block_text: str):
#     """
#     Parse lines: relation("Subject","Object")
#     - relation may include spaces/underscores
#     - subject/object must be double-quoted (handles commas safely)
#     """
#     triples = []
#     line_re = re.compile(r'^\s*([A-Za-z][A-Za-z0-9_ ]*?)\s*\(\s*"([^"]+)"\s*,\s*"([^"]+)"\s*\)\s*$')
#     for raw in block_text.splitlines():
#         line = raw.strip()
#         if not line or line.startswith("#"):
#             continue
#         m = line_re.match(line)
#         if not m:
#             # skip non-conforming lines
#             continue
#         rel, sub, obj = (m.group(1).strip(), m.group(2).strip(), m.group(3).strip())
#         triples.append({"sub": sub, "rel": rel, "obj": obj})
#     return triples


# # -------- Utilities: JSONL I/O --------
# def read_jsonl(path, max_items=None):
#     count = 0
#     with open(path, "r", encoding="utf-8") as f:
#         for line in f:
#             line = line.strip()
#             if not line:
#                 continue
#             yield json.loads(line)
#             count += 1
#             if max_items is not None and count >= max_items:
#                 break

# def write_jsonl(path, records):
#     with open(path, "w", encoding="utf-8") as f:
#         for rec in records:
#             f.write(json.dumps(rec, ensure_ascii=False))
#             f.write("\n")


# # -------- Orchestration: process file --------
# def run_pipeline(
#     input_jsonl_path: str,
#     ontology_json_path: str,
#     output_jsonl_path: str,
#     max_items: int = 4,
#     max_new_tokens: int = 768,
#     temperature: float = 0.3,
#     verbose: bool = True
# ):
#     # Load ontology once
#     with open(ontology_json_path, "r", encoding="utf-8") as f:
#         ontology = json.load(f)

#     # Load model
#     generator, tokenizer = setup_model("mistralai/Mistral-7B-Instruct-v0.3")

#     out_records = []
#     t0 = time.time()

#     for idx, item in enumerate(read_jsonl(input_jsonl_path, max_items=max_items), start=1):
#         sent_id = item.get("id")
#         sent    = item.get("sent", "")

#         # Build prompt
#         prompt_text = build_reason_then_extract_prompt(
#             ontology_json=ontology,
#             test_sentence=sent,
#             worked_example=None,       # keep None; you can inject a worked example if you want
#             allow_light_norm=True,
#             relation_cues_text=None    # keep None; can inject paraphrase cues later
#         )

#         # Inference
#         gen_t0 = time.time()
#         model_output = run_single_inference_chat(generator, tokenizer, prompt_text, max_new_tokens=max_new_tokens, temperature=temperature)
#         gen_dt = time.time() - gen_t0

#         # Extract & parse
#         final_block = extract_final_triples_block(model_output)
#         triples = parse_triples_block(final_block)

#         if verbose:
#             print(f"\n[{idx}] ID={sent_id}")
#             print(f"Sentence: {sent}")
#             print(f"Generated in {gen_dt:.2f}s | Triples: {len(triples)}")
#             if triples:
#                 print("Sample:", triples[0])

#         out_records.append({"id": sent_id, "triples": triples})

#     # Write output JSONL
#     write_jsonl(output_jsonl_path, out_records)
#     print(f"\n✅ Done. Wrote {len(out_records)} lines to: {output_jsonl_path} | Total time: {time.time()-t0:.1f}s")


# # =========================
# # Configure your paths here
# # =========================

# # Example paths (edit these to your files)
# INPUT_JSONL  = "/path/to/your/input_texts.jsonl"   # the file with your 3 test lines
# ONTOLOGY_JSON = "/path/to/your/university_ontology.json"
# OUTPUT_JSONL  = "/path/to/output_triples.jsonl"

# # Process only first N lines for debugging
# MAX_ITEMS = 4   # set to 20 for a larger preview, or None for full run

# # Run the pipeline
# # (Uncomment the line below after setting valid paths)
# # run_pipeline(INPUT_JSONL, ONTOLOGY_JSON, OUTPUT_JSONL, max_items=MAX_ITEMS, max_new_tokens=768, temperature=0.25, verbose=True)


In [4]:
# love it. here’s a **clean, modular pipeline**—each step in its own block/function so you can debug or swap parts easily.

# ---

# # Block 0 — Imports & Config

# ```python
# # Block 0 — Imports & Config

# import os, json, re, time
# import torch
# from textwrap import dedent
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# ```

# ---

# # Block 1 — Model Setup

# ```python
# # Block 1 — Model Setup

# def setup_model(model_id="mistralai/Mistral-7B-Instruct-v0.3"):
#     """
#     Returns (generator, tokenizer) ready for inference with chat template.
#     """
#     torch.backends.cudnn.benchmark = True
#     tokenizer = AutoTokenizer.from_pretrained(model_id)
#     model = AutoModelForCausalLM.from_pretrained(
#         model_id,
#         device_map="auto",
#         torch_dtype=torch.float16,
#     )
#     model.config.use_cache = False

#     generator = pipeline(
#         "text-generation",
#         model=model,
#         tokenizer=tokenizer,
#         device_map="auto"
#     )
#     return generator, tokenizer
# ```

# ---

# # Block 2 — Prompt Builder (Reason → Verify → Final Triples)

# ```python
# # Block 2 — Prompt Builder (Reason → Verify → Final Triples)

# def _concept_label(ontology_json, qid):
#     return next((c["label"] for c in ontology_json.get("concepts", []) if c.get("qid") == qid), "")

# def format_ontology_concepts(ontology_json):
#     return ", ".join(c["label"] for c in ontology_json.get("concepts", []))

# def format_ontology_relations(ontology_json):
#     lines = []
#     for r in ontology_json.get("relations", []):
#         dom = _concept_label(ontology_json, r.get("domain"))
#         rng = _concept_label(ontology_json, r.get("range"))
#         lines.append(f'- {r["label"]}({dom},{rng})')
#     return "\n".join(lines)

# def build_reason_then_extract_prompt(
#     ontology_json,
#     test_sentence: str,
#     worked_example: dict | None = None,
#     allow_light_norm: bool = True,
#     relation_cues_text: str | None = None,   # optional Step 0 paraphrase cues
# ) -> str:
#     """
#     Single-output-block design: ### FINAL TRIPLES includes
#     - ontology-mapped relations (preferred)
#     - plus any other sentence-backed SPO relations (open-set)
#     """
#     concepts_line   = format_ontology_concepts(ontology_json)
#     relations_block = format_ontology_relations(ontology_json)
#     norm_note = "normalize only trivial cases (e.g., Japanese→Japan)" if allow_light_norm else "copy spans verbatim; no normalization"

#     step0_block = ""
#     if relation_cues_text:
#         step0_block = dedent(f"""
#         Step 0 (Paraphrase expansion for this sentence):
#         {relation_cues_text.strip()}
#         """)

#     example_block = ""
#     if worked_example:
#         example_block = dedent(f"""
#         ### WORKED EXAMPLE
#         Sentence:
#         {worked_example["sentence"]}

#         Step 1 (Entities & types):
#         {worked_example["step1"]}

#         Step 2 (Verify relations):
#         {worked_example["step2"]}

#         ### FINAL TRIPLES
#         {worked_example["final_triples"].strip()}
#         """).strip()

#     rules = dedent("""
#     RULES:
#     - Prefer relation labels from ONTOLOGY RELATIONS when they match the evidence.
#     - If the sentence clearly states another SPO relation NOT in the ontology, still include it using a concise predicate phrase from the sentence (do NOT invent facts).
#     - Resolve simple coreference (e.g., "the university", "it", "this college" → the named university in this sentence).
#     - Quote/mention evidence in Step 2 for each emitted triple.
#     - Output one triple per line, format: predicate("Subject","Object").
#     - Base decisions ONLY on the TEST SENTENCE.
#     - Avoid duplicates; apply light normalization only where unambiguous.
#     """)

#     prompt = dedent(f"""
#     TASK: Extract all SPO triples that the TEST SENTENCE clearly supports.
#     - Use the ontology to guide relation labeling when possible.
#     - {norm_note}.
#     - If a valid SPO fact has no matching ontology label, still output it (single FINAL TRIPLES list).

#     ONTOLOGY CONCEPTS:
#     {concepts_line}

#     ONTOLOGY RELATIONS (argument types):
#     {relations_block}

#     {rules}

#     {step0_block}

#     {example_block if example_block else ""}

#     ### TEST SENTENCE
#     "{test_sentence}"

#     Step 1 (Entities & types):
#     Step 2 (Verify relations): For each candidate triple, quote or mention the trigger phrase and spans.

#     ### FINAL TRIPLES
#     # one triple per line; include ontology-mapped AND extra sentence-backed relations
#     """).strip()

#     return prompt
# ```

# ---

# # Block 3 — Single Inference (chat template; continuation-only)

# ```python
# # Block 3 — Single Inference (chat template; continuation-only)

# def generate_triples_text(generator, tokenizer, prompt_text: str,
#                           max_new_tokens: int = 768, temperature: float = 0.3) -> str:
#     """
#     Calls the model once. Returns the full generated continuation (reasoning + FINAL TRIPLES).
#     """
#     chat = [
#         {"role": "system", "content": "You are a precise information-extraction model. Follow instructions carefully."},
#         {"role": "user", "content": prompt_text}
#     ]
#     formatted = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

#     out = generator(
#         formatted,
#         max_new_tokens=max_new_tokens,
#         temperature=temperature,
#         top_p=0.9,
#         do_sample=True,
#         return_full_text=False,
#         truncation=False,
#         eos_token_id=tokenizer.eos_token_id,
#         pad_token_id=tokenizer.eos_token_id,
#     )
#     return out[0]["generated_text"] if isinstance(out[0], dict) else out[0]
# ```

# ---

# # Block 4 — Extract & Parse `FINAL TRIPLES`

# ```python
# # Block 4 — Extract & Parse `FINAL TRIPLES`

# FINAL_HEADER_REGEX = r'^\s*#{2,}\s*FINAL\s+TRIPLES\s*$'  # matches ## or ### FINAL TRIPLES

# def extract_final_triples_block(model_output_text: str) -> str:
#     """
#     Return the text after the '##/### FINAL TRIPLES' header (until next header or end).
#     """
#     header = re.search(FINAL_HEADER_REGEX, model_output_text, re.IGNORECASE | re.MULTILINE)
#     if not header:
#         header = re.search(r'FINAL\s+TRIPLES\s*[:\-]*\s*$', model_output_text, re.IGNORECASE | re.MULTILINE)
#         if not header:
#             return ""
#     start = header.end()
#     tail = model_output_text[start:].strip()
#     nxt = re.search(r'^\s*#{2,}\s+[A-Z].*$', tail, re.MULTILINE)
#     if nxt:
#         tail = tail[:nxt.start()].strip()
#     return tail

# def parse_triples_block(block_text: str):
#     """
#     Parse lines: relation("Subject","Object")
#     Keeps both ontology and non-ontology predicates (open-set allowed).
#     """
#     triples = []
#     line_re = re.compile(r'^\s*([A-Za-z][A-Za-z0-9_ ]*?)\s*\(\s*"([^"]+)"\s*,\s*"([^"]+)"\s*\)\s*$')
#     for raw in block_text.splitlines():
#         line = raw.strip()
#         if not line or line.startswith("#"):
#             continue
#         m = line_re.match(line)
#         if not m:
#             continue
#         rel, sub, obj = (m.group(1).strip(), m.group(2).strip(), m.group(3).strip())
#         triples.append({"sub": sub, "rel": rel, "obj": obj})
#     return triples
# ```

# ---

# # Block 5 — JSONL I/O Helpers

# ```python
# # Block 5 — JSONL I/O Helpers

# def read_jsonl(path, max_items: int | None = None):
#     """
#     Yields JSON objects from a .jsonl file.
#     If max_items is set, stops after that many records (for debugging).
#     """
#     count = 0
#     with open(path, "r", encoding="utf-8") as f:
#         for line in f:
#             line = line.strip()
#             if not line:
#                 continue
#             yield json.loads(line)
#             count += 1
#             if max_items is not None and count >= max_items:
#                 break

# def write_jsonl(path, records):
#     with open(path, "w", encoding="utf-8") as f:
#         for rec in records:
#             f.write(json.dumps(rec, ensure_ascii=False))
#             f.write("\n")
# ```

# ---

# # Block 6 — Orchestrator (loop inputs → Steps 1–4 → output)

# ```python
# # Block 6 — Orchestrator (loop inputs → Steps 1–4 → output)

# def run_pipeline(
#     input_jsonl_path: str,
#     ontology_json_path: str,
#     output_jsonl_path: str,
#     max_items: int = 4,           # ← process only first N rows (easy debugging)
#     max_new_tokens: int = 768,
#     temperature: float = 0.3,
#     verbose: bool = True
# ):
#     # Load ontology once
#     with open(ontology_json_path, "r", encoding="utf-8") as f:
#         ontology = json.load(f)

#     # Setup model once
#     generator, tokenizer = setup_model("mistralai/Mistral-7B-Instruct-v0.3")

#     out_records = []
#     t0 = time.time()

#     for idx, item in enumerate(read_jsonl(input_jsonl_path, max_items=max_items), start=1):
#         sent_id = item.get("id")
#         sent    = item.get("sent", "")

#         # Step 2 — Build prompt
#         prompt_text = build_reason_then_extract_prompt(
#             ontology_json=ontology,
#             test_sentence=sent,
#             worked_example=None,
#             allow_light_norm=True,
#             relation_cues_text=None
#         )

#         # Step 3 — Generate
#         t_gen0 = time.time()
#         model_output = generate_triples_text(
#             generator, tokenizer, prompt_text,
#             max_new_tokens=max_new_tokens, temperature=temperature
#         )
#         gen_time = time.time() - t_gen0

#         # Step 4 — Extract + Parse
#         final_block = extract_final_triples_block(model_output)
#         triples = parse_triples_block(final_block)

#         if verbose:
#             print(f"\n[{idx}] ID={sent_id}")
#             print(f"Sentence: {sent}")
#             print(f"Generated in {gen_time:.2f}s | Triples: {len(triples)}")
#             if triples:
#                 print("Example triple:", triples[0])

#         out_records.append({"id": sent_id, "triples": triples})

#     # Write output
#     write_jsonl(output_jsonl_path, out_records)
#     print(f"\n✅ Done. Wrote {len(out_records)} lines to: {output_jsonl_path} | Total time: {time.time()-t0:.1f}s")
# ```

# ---

# # Block 7 — Example Run (set paths & N)

# ```python
# # Block 7 — Example Run (edit these paths)

# INPUT_JSONL   = "/path/to/input_university_texts.jsonl"     # your 3 sample lines as JSONL
# ONTOLOGY_JSON = "/path/to/university_ontology.json"         # your ontology JSON
# OUTPUT_JSONL  = "/path/to/output_triples.jsonl"             # where to write results

# MAX_ITEMS = 4   # change to 20 (or None for full file) to control batches during debugging

# # Uncomment to execute:
# # run_pipeline(INPUT_JSONL, ONTOLOGY_JSON, OUTPUT_JSONL,
# #              max_items=MAX_ITEMS, max_new_tokens=768, temperature=0.25, verbose=True)
# ```

# ---

# ### why this layout is nice to debug

# * You can **unit-test** each block separately:

#   * Block 2: print `prompt_text`
#   * Block 3: print the **raw model output**
#   * Block 4: print the **extracted block** and **parsed triples**
# * **Swap** the model easily (change the `setup_model` param).
# * **Scale** later: add fusion/validation after Block 4 without touching earlier blocks.

# want me to add a tiny **validation Block 8** next (ontology-conformance flag + entity-present-in-sentence check)?
